In [1]:
import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [2]:
import os
import pandas as pd
import numpy as np
import glob
from utils import LoadData, AbsPaths
from datapackage import PackageCARACTERÍSTICAS

In [247]:
def select_columns(file_name = "atp_matches_historic", cols:str=['tourney_id','round','winner_id',
                                                                 'winner_rank', 'winner_rank_points', 'loser_id',
                                                             'loser_rank', 'loser_rank_points','score','minutes'],
                   path_file_name=None, path_to_save = None):
    if path_file_name is None:
        path_file_name=AbsPaths().get_abs_path_folder(folder_name="interim")+ f"{file_name}" + ".csv"
    data=pd.read_csv(path_file_name)
    data=data.loc[:,cols]
    return data

In [4]:
def tourney_table(file_name:str=None, path_to_save = None):
    data=select_columns(cols=['tourney_id', 'tourney_name', 'surface','tourney_date']).drop_duplicates()
    if path_to_save is None:
        path_to_save = AbsPaths().get_abs_path_folder(folder_name='interim')
    if file_name is None:
        file_name= 'TourneyTable'
    data.to_csv(path_to_save + f"{file_name}.csv", index=False)

In [5]:
def GetAuxiliarDataHub(link='https://datahub.io/sports-data/atp-world-tour-tennis-data/datapackage.json'):
    package = Package(link)
    return package

def SaveAuxiliarDataHub(path_to_save, table_to_extract):
    package=GetAuxiliarDataHub()
    id_players=package.resource_names.index(table_to_extract)
    Data=pd.DataFrame(package.resources[id_players].read(), columns=package.resources[id_players].headers)
    if path_to_save is None:
        path_to_save = AbsPaths().get_abs_path_folder(folder_name='external')
    Data.to_csv(path_to_save + f"{table_to_extract}.csv", index=False)

    
def OpenAuxiliarData(name:str='player_overviews_unindexed', path:str=None):
    if path is None:
        path= AbsPaths().get_abs_path_folder(folder_name='external')
    if not os.path.exists(path+ f"{name}.csv"):
        auxiliarData=SaveAuxiliarDataHub(path, name)
    AuxiliarData=pd.read_csv(path+ f"{name}.csv")
    return AuxiliarData


In [239]:
def PlayerTable(file_name:str=None, path_to_save = None):
    data=select_columns(cols=['tourney_date','winner_id','winner_name','winner_ht','winner_age','winner_ioc']).rename(columns={'winner_id':'id','winner_name':'name','winner_ht':'ht', 'winner_age':'age','winner_ioc':'ioc'})                          
    data2=select_columns(cols=['tourney_date','loser_id','loser_name','loser_ht','loser_age','loser_ioc']).rename(columns={'loser_id':'id','loser_name':'name','loser_ht':'ht', 'loser_age':'age','loser_ioc':'ioc'})
    players=pd.concat([data, data2]).drop_duplicates(subset=['id','name','ht','ioc']).reset_index()    
    for i in players.index:
        players.loc[i,'birthdate']=(pd.to_datetime(players.loc[i,'tourney_date'], format='%Y%m%d')-pd.Timedelta(str(players.loc[i,'age'])+'Y')).strftime('%Y-%m-%d')
    
    players=AddCharacteristic_PlayerTable(players, 'weight_kg')
    players=ReviewHt_PlayerTable(players)
    
    if path_to_save is None:
        path_to_save = AbsPaths().get_abs_path_folder(folder_name='interim')
    if file_name is None:
        file_name= 'PlayerTable'
    players.loc[:,['id','name','birthdate','weight_kg','ht','ioc']].to_csv(path_to_save + f"{file_name}.csv", index=False)

def AddCharacteristic_PlayerTable(players, characteristic):
    AuxiliarData=OpenAuxiliarData()
    AuxiliarData.loc[:,'name']=AuxiliarData.loc[:,'first_name'].str.replace('-',' ').str.replace('.','')+' '+AuxiliarData.loc[:,'last_name'].str.replace('-',' ').str.replace('.','').str.title()
    for i in players.index:
        if len(AuxiliarData[AuxiliarData.name==players.loc[i,'name']][AuxiliarData.flag_code==players.loc[i,'ioc']][characteristic])>0:
            players.loc[i, characteristic]=AuxiliarData[AuxiliarData.name==players.loc[i,'name']][AuxiliarData.flag_code==players.loc[i,'ioc']][characteristic].values[0] 
        else:
            try:
                players.loc[i, characteristic]=AuxiliarData[AuxiliarData.name==players.loc[i,'name']][characteristic].values[0] 
            except:pass
    return players

def ReviewHt_PlayerTable(players):
    AuxiliarData=OpenAuxiliarData()
    AuxiliarData.loc[:,'name']=AuxiliarData.loc[:,'first_name'].str.replace('-',' ').str.replace('.','')+' '+AuxiliarData.loc[:,'last_name'].str.replace('-',' ').str.replace('.','').str.title()
    for i in players[players.ht.isna()].index:
        if len(AuxiliarData[AuxiliarData.name==players.loc[i,'name']][AuxiliarData.flag_code==players.loc[i,'ioc']]['height_cm'])>0:
            players.loc[i, 'ht']=AuxiliarData[AuxiliarData.name==players.loc[i,'name']][AuxiliarData.flag_code==players.loc[i,'ioc']]['height_cm'].values[0] 
        else:
            try:
                players.loc[i, 'ht']=AuxiliarData[AuxiliarData.name==players.loc[i,'name']]['height_cm'].values[0] 
            except:pass
    players.loc[players.ht.isna(),'ht']=players.ht.mean()
    return players

In [240]:
players=PlayerTable()

/tmp/ipykernel_11585/1450800076.py:6: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  players.loc[i,'birthdate']=(pd.to_datetime(players.loc[i,'tourney_date'], format='%Y%m%d')-pd.Timedelta(str(players.loc[i,'age'])+'Y')).strftime('%Y-%m-%d')
/tmp/ipykernel_11585/1450800076.py:6: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  players.loc[i,'birthdate']=(pd.to_datetime(players.loc[i,'tourney_date'], format='%Y%m%d')-pd.Timedelta(str(players.loc[i,'age'])+'Y')).strftime('%Y-%m-%d')
/tmp/ipykernel_11585/1450800076.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  AuxiliarData.loc[:,'name']=AuxiliarData.loc[:,'first_name'].str.replace('-',' ').str.replace('.','')+' '+Auxiliar

In [308]:
def TableMatches(file_name:str=None, path_to_save = None):
    data=select_columns()
    Nan=NanColumns(data)
    data=CleanNaNData(data, Nan)
    return data

In [299]:
Nandd

Index(['winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'minutes'],
      dtype='object')

In [309]:
data=TableMatches()

In [311]:
data[data.minutes.isna()]

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes


In [307]:
def NanColumns(data):
    NanCols=data.keys()[data.isna().sum()>0]
    return NanCols

def CleanNaNData(data, Nan):
    if 'minutes' in Nan:
        data=CleanMinutes(data)
    Nan=Nan.drop('minutes')
    #data=CleanProperties(data, Nan)
    return data

def CleanMinutes(data):
    data.loc[data.score=='W/O','minutes']=0
    data.loc[data.minutes.isna(), 'minutes']=data.loc[data.minutes.isna(),'score'].str.split(' ').apply(lambda x: len(x) * 35)
    return data

def CleanProperties(data, Nan):
    for j in Nan:
        for k in data[data[j].isna()].index
            if 'winner' in j:
                data.loc[k,j]=data.loc[:data[data[j].isna()].index[0]-1,:][data.winner_id==data[data[j].isna()]['winner_id'].values[0]].dropna().iloc[-1][j]

In [ ]:
## for j in Nan:
    for k in data[data[j].isna()].index:
        if 'winner' in j:
            data.loc[k,j]=data.loc[:data[data[j].isna()].index[0]-1,:][data.winner_id==data[data[j].isna()]['winner_id'].values[0]].dropna().iloc[-1][j]
        if 'loser' in j:
            data.loc[k,j]=data.loc[:data[data[j].isna()].index[0]-1,:][data.loser_id==data[data[j].isna()]['loser_id'].values[0]].dropna().iloc[-1][j]

In [354]:
data.keys()[data.isna().sum()>0]

Index(['loser_rank', 'loser_rank_points'], dtype='object')

In [352]:
data.dropna()

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
0,2000-560,R128,101736,1.0,3754.0,103297.0,432.0,53.0,6-4 6-2 6-0,89.0
1,2000-560,R128,103096,37.0,928.0,103292.0,26.0,1067.0,6-3 6-4 6-2,105.0
2,2000-560,R128,102271,41.0,889.0,103082.0,92.0,448.0,6-3 6-0 4-6 6-7(3) 6-4,214.0
3,2000-560,R128,101965,21.0,1227.0,102776.0,51.0,794.0,6-3 2-6 6-4 6-1,116.0
4,2000-560,R128,103324,63.0,642.0,102735.0,311.0,98.0,6-4 6-4 4-6 6-3,147.0
...,...,...,...,...,...,...,...,...,...,...
8123,2015-520,QF,104542,15.0,2045.0,105453.0,5.0,5220.0,6-1 6-4 4-6 3-6 6-3,225.0
8124,2015-520,QF,104527,9.0,3845.0,103819.0,2.0,9235.0,6-4 6-3 7-6(4),129.0
8125,2015-520,SF,104925,1.0,13845.0,104918.0,3.0,7040.0,6-3 6-3 5-7 5-7 6-1,249.0
8126,2015-520,SF,104527,9.0,3845.0,104542.0,15.0,2045.0,6-3 6-7(1) 7-6(3) 6-4,226.0


In [313]:
data[data.winner_rank.isna()][j]=

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
3854,2007-540,R128,103017,NaN,NaN,103835.0,27.0,987.0,6-3 7-6(6) 6-1,147.0
3896,2007-540,R64,103017,NaN,NaN,102148.0,70.0,551.0,6-4 6-3 6-4,115.0


In [330]:
data[data[j].isna()]['tourney_id'].values[0]

'2007-540'

In [341]:
'winner' in j

True

In [344]:
data[data[j].isna()]['winner_id'].values[0]

103017

In [348]:
data.loc[:data[data[j].isna()].index[0]-1,:][data.winner_id==data[data[j].isna()]['winner_id'].values[0]].dropna().iloc[-1][j]

/tmp/ipykernel_11585/518085788.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data.loc[:data[data[j].isna()].index[0]-1,:][data.winner_id==data[data[j].isna()]['winner_id'].values[0]].dropna().iloc[-1][j]


187.0

In [338]:
if 'winner' in j:
    data.loc[:data[data[j].isna()].index[0]-1,:][data.winner_id==data[data[j].isna()]]

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
0,2000-560,R128,101736,1.0,3754.0,103297.0,432.0,53.0,6-4 6-2 6-0,89.0
1,2000-560,R128,103096,37.0,928.0,103292.0,26.0,1067.0,6-3 6-4 6-2,105.0
2,2000-560,R128,102271,41.0,889.0,103082.0,92.0,448.0,6-3 6-0 4-6 6-7(3) 6-4,214.0
3,2000-560,R128,101965,21.0,1227.0,102776.0,51.0,794.0,6-3 2-6 6-4 6-1,116.0
4,2000-560,R128,103324,63.0,642.0,102735.0,311.0,98.0,6-4 6-4 4-6 6-3,147.0
...,...,...,...,...,...,...,...,...,...,...
3849,2007-540,R128,104571,16.0,1570.0,105208.0,82.0,512.0,3-6 6-4 6-3 6-2,117.0
3850,2007-540,R128,103720,19.0,1330.0,104171.0,194.0,220.0,7-5 6-3 7-5,123.0
3851,2007-540,R128,104620,65.0,565.0,104198.0,58.0,593.0,4-6 7-6(5) 6-4 6-4,151.0
3852,2007-540,R128,103018,57.0,598.0,104229.0,106.0,381.0,6-3 6-4 2-1 RET,94.0


In [327]:
data.loc[:,data.tourney_id==data[j].isna()['tourney_id'].values[0]]

KeyError: 'tourney_id'

In [324]:
j=Nan[0]
data[data[j].isna()]

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
3854,2007-540,R128,103017,NaN,NaN,103835.0,27.0,987.0,6-3 7-6(6) 6-1,147.0
3896,2007-540,R64,103017,NaN,NaN,102148.0,70.0,551.0,6-4 6-3 6-4,115.0


In [319]:
j

'winner_rank'

In [314]:
'winner' in Nan

False

In [315]:
Nan

Index(['winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'minutes'],
      dtype='object')

In [300]:
data[data.score=='W/O'].loc[:,'minutes']=0

/tmp/ipykernel_11585/4014879879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data.score=='W/O'].loc[:,'minutes']=0
/tmp/ipykernel_11585/4014879879.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data[data.score=='W/O'].loc[:,'minutes']=0


In [305]:
data.loc[data.score=='W/O','minutes']=0

In [306]:
data[data.score=='W/O']

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
1878,2003-580,R32,102783,36.0,945.0,103498.0,3.0,2860.0,W/O,0.0
2144,2004-560,R16,103819,1.0,5910.0,102318.0,18.0,1310.0,W/O,0.0
2619,2005-520,R64,103292,6.0,2440.0,104098.0,162.0,272.0,W/O,0.0
2658,2005-520,R16,103084,10.0,1745.0,103017.0,28.0,1130.0,W/O,0.0
3003,2005-560,R64,103454,58.0,665.0,102562.0,25.0,1230.0,W/O,0.0
3922,2007-540,R16,103819,1.0,7290.0,103163.0,10.0,1960.0,W/O,0.0
5044,2009-540,R64,104542,9.0,3510.0,104620.0,63.0,951.0,W/O,0.0
5166,2010-540,R64,104468,32.0,1305.0,105011.0,74.0,672.0,W/O,0.0
5435,2010-580,R32,103997,86.0,573.0,104022.0,20.0,1690.0,W/O,0.0
6092,2011-520,QF,104925,2.0,11665.0,104926.0,49.0,960.0,W/O,0.0


In [ ]:
CleanNaNData(data, Nan)

In [295]:
data[data.minutes.isna()].loc[:,'minutes']

1878   NaN
2144   NaN
2619   NaN
2658   NaN
3003   NaN
        ..
7871   NaN
7872   NaN
7873   NaN
8074   NaN
8105   NaN
Name: minutes, Length: 279, dtype: float64

In [294]:
data[data.minutes.isna()].loc[:,'score'].str.split(' ').apply(lambda x: len(x) * 35)

1878     35
2144     35
2619     35
2658     35
3003     35
       ... 
7871    105
7872    105
7873    140
8074     70
8105     70
Name: score, Length: 279, dtype: int64

In [284]:
data[data.minutes.isna()].loc[:,'score'].str.split(' ').apply(lambda x: len(x) * 35)

1878     35
2144     35
2619     35
2658     35
3003     35
       ... 
7871    105
7872    105
7873    140
8074     70
8105     70
Name: score, Length: 279, dtype: int64

In [281]:
len(data.loc[0,'score'].split(' '))*35

105

In [278]:
data

,tourney_id,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
0,2000-560,R128,101736,1.0,3754.0,103297.0,432.0,53.0,6-4 6-2 6-0,89.0
1,2000-560,R128,103096,37.0,928.0,103292.0,26.0,1067.0,6-3 6-4 6-2,105.0
2,2000-560,R128,102271,41.0,889.0,103082.0,92.0,448.0,6-3 6-0 4-6 6-7(3) 6-4,214.0
3,2000-560,R128,101965,21.0,1227.0,102776.0,51.0,794.0,6-3 2-6 6-4 6-1,116.0
4,2000-560,R128,103324,63.0,642.0,102735.0,311.0,98.0,6-4 6-4 4-6 6-3,147.0
...,...,...,...,...,...,...,...,...,...,...
8123,2015-520,QF,104542,15.0,2045.0,105453.0,5.0,5220.0,6-1 6-4 4-6 3-6 6-3,225.0
8124,2015-520,QF,104527,9.0,3845.0,103819.0,2.0,9235.0,6-4 6-3 7-6(4),129.0
8125,2015-520,SF,104925,1.0,13845.0,104918.0,3.0,7040.0,6-3 6-3 5-7 5-7 6-1,249.0
8126,2015-520,SF,104527,9.0,3845.0,104542.0,15.0,2045.0,6-3 6-7(1) 7-6(3) 6-4,226.0


In [86]:
data=select_columns()
NanCols=NanColumns()

In [262]:
data

NameError: name 'data' is not defined

In [238]:
AuxiliarData=OpenAuxiliarData()

In [220]:
data=select_columns()

In [221]:
atpplayerdata=SaveAuxiliarDataHub()

In [229]:
atpplayerdata[(atpplayerdata.first_name=='Andre' )&  (atpplayerdata.flag_code=='USA') ]


,player_id,player_slug,first_name,last_name,player_url,flag_code,residence,birthplace,birthdate,birth_year,birth_month,birth_day,turned_pro,weight_lbs,weight_kg,height_ft,height_inches,height_cm,handedness,backhand
53,a092,andre-agassi,Andre,Agassi,http://www.atpworldtour.com/en/players/andre-a...,USA,"Las Vegas, Nevada, USA","Las Vegas, Nevada, USA",1970.04.29,1970.0,4,29,1986.0,177.0,80.0,"5'11""",71.0,180.0,Right-Handed,Two-Handed Backhand
2407,db49,andre-dome,Andre,Dome,http://www.atpworldtour.com/en/players/andre-d...,USA,"Arroyo Grande, CA",Samosir Island,1990.08.13,1990.0,8,13,NaN,170.0,77.0,"5'11""",71.0,180.0,Right-Handed,Two-Handed Backhand


In [224]:
players

,id,name,ht,ioc
0,101736.0,Andre Agassi,180.0,USA
1,103096.0,Arnaud Clement,173.0,FRA
2,102271.0,Hicham Arazi,175.0,MAR
3,101965.0,Wayne Ferreira,185.0,RSA
4,103324.0,Arnaud Di Pasquale,185.0,FRA
...,...,...,...,...
8005,105325.0,Germain Gigounon,NaN,BEL
8016,111460.0,Quentin Halys,NaN,FRA
8027,111506.0,Maxime Hamou,NaN,FRA
8041,105872.0,Christian Lindell,NaN,SWE


In [90]:
set(data[data.winner_rank==1]['winner_name'])

{'Andre Agassi',
 'Andy Roddick',
 'Gustavo Kuerten',
 'Lleyton Hewitt',
 'Novak Djokovic',
 'Rafael Nadal',
 'Roger Federer'}

In [85]:
set(data[data['winner_ht'].isna()].loc[:,'winner_name'])

{'Alejandro Gonzalez',
 'Aleksandr Nedovyesov',
 'Alexander Zverev',
 'Andrea Arnaboldi',
 'Ante Pavic',
 'Austin Krajicek',
 'Axel Michon',
 'Blaz Rola',
 'Borna Coric',
 'Chris Eaton',
 'Dennis Novikov',
 'Diego Sebastian Schwartzman',
 'Dominic Thiem',
 'Facundo Bagnis',
 'Guido Pella',
 'Hyeon Chung',
 'Inigo Cervantes Huegun',
 'Ivan Sergeyev',
 'Jan Lennard Struff',
 'Jiri Vesely',
 'Joao Sousa',
 'Kyle Edmund',
 'Laurent Recouderc',
 'Lee Childs',
 'Liam Broady',
 'Lucas Pouille',
 'Luke Saville',
 'Martin Fischer',
 'Maxime Teixeira',
 'Nick Kyrgios',
 'Nikoloz Basilashvili',
 'Pablo Carreno Busta',
 'Pierre Hugues Herbert',
 'Roberto Bautista Agut',
 'Ruben Bemelmans',
 'Samuel Groth',
 'Sergei Bubka',
 'Thanasi Kokkinakis',
 'Tim Puetz',
 'Yoshihito Nishioka'}

In [ ]:
'tourney_id', 'tourney_name', 'surface','tourney_date', 'match_num','winner_id','winner_name','winner_ht', 'winner_age','winner_rank', 'winner_rank_points', 'loser_id','loser_name','loser_ht', 'loser_age','loser_rank', 'loser_rank_points','minutes'




In [ ]:
jugador, torneo, cancha, fecha, pj, pg, pp, altura, edad, rank_inicial, minutos

In [7]:
data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,2000-560,US Open,Hard,128,G,20000828.0,1.0,101736,1.0,NaN,...,3.0,3.0,8.0,90.0,52.0,30.0,13.0,12.0,5.0,12.0
1,2000-560,US Open,Hard,128,G,20000828.0,2.0,103096,NaN,NaN,...,3.0,5.0,4.0,84.0,53.0,34.0,12.0,13.0,7.0,12.0
2,2000-560,US Open,Hard,128,G,20000828.0,3.0,102271,NaN,NaN,...,13.0,7.0,3.0,167.0,95.0,58.0,32.0,23.0,10.0,19.0
3,2000-560,US Open,Hard,128,G,20000828.0,4.0,101965,NaN,NaN,...,10.0,7.0,7.0,103.0,61.0,41.0,13.0,17.0,5.0,13.0
4,2000-560,US Open,Hard,128,G,20000828.0,5.0,103324,NaN,NaN,...,5.0,3.0,4.0,131.0,93.0,57.0,19.0,19.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2015-520,Roland Garros,Clay,128,G,20150524.0,123.0,104542,14.0,NaN,...,7.0,5.0,6.0,160.0,106.0,71.0,23.0,22.0,13.0,19.0
8124,2015-520,Roland Garros,Clay,128,G,20150524.0,124.0,104527,8.0,NaN,...,4.0,7.0,0.0,112.0,71.0,48.0,23.0,16.0,5.0,8.0
8125,2015-520,Roland Garros,Clay,128,G,20150524.0,125.0,104925,1.0,NaN,...,4.0,12.0,1.0,149.0,98.0,63.0,28.0,24.0,13.0,19.0
8126,2015-520,Roland Garros,Clay,128,G,20150524.0,126.0,104527,8.0,NaN,...,17.0,8.0,2.0,147.0,102.0,73.0,25.0,21.0,12.0,15.0


In [6]:
data.keys()

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

In [35]:
for i in data.keys():
    print(i +':')
    print(set(data[i]))

tourney_id:
{'2015-540', '2012-560', '2003-540', '2001-520', '2015-560', '2013-540', '2001-560', '2008-560', '2005-580', '2001-580', '2005-540', '2004-560', '2011-540', '2008-540', '2012-520', '2002-560', '2006-520', '2013-560', '2001-540', '2007-560', '2007-520', '2006-540', '2009-580', '2000-520', '2012-580', '2002-520', '2011-520', '2014-580', '2005-520', '2011-580', '2004-580', '2000-580', '2008-580', '2005-560', '2010-520', '2003-520', '2013-520', '2003-560', '2002-540', '2006-580', '2007-580', '2014-540', '2010-580', '2006-560', '2004-520', '2013-580', '2000-560', '2010-560', '2014-560', '2002-580', '2009-560', '2015-520', '2014-520', '2007-540', '2009-540', '2000-540', '2012-540', '2015-580', '2004-540', '2008-520', '2010-540', '2009-520', '2003-580', '2011-560'}
tourney_name:
{'US Open', 'Australian Open', 'Wimbledon', 'Roland Garros'}
surface:
{'Grass', 'Clay', 'Hard'}
tourney_date:
{20070528.0, 20010625.0, 20010115.0, 20090119.0, 20130826.0, 20050829.0, 20080525.0, 20100621.0

In [27]:
data[data.score=='W/O'].loc[:,data.columns[10:35]]

,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,...,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn
1878,Rainer Schuettler,R,180.0,GER,26.718686,36.0,945.0,103498.0,3.0,NaN,...,3.0,2860.0,W/O,5.0,R32,NaN,NaN,NaN,NaN,NaN
2144,Roger Federer,R,185.0,SUI,23.060917,1.0,5910.0,102318.0,16.0,NaN,...,18.0,1310.0,W/O,5.0,R16,NaN,NaN,NaN,NaN,NaN
2619,Gaston Gaudio,R,175.0,ARG,26.453114,6.0,2440.0,104098.0,NaN,NaN,...,162.0,272.0,W/O,5.0,R64,NaN,NaN,NaN,NaN,NaN
2658,Guillermo Canas,R,185.0,ARG,27.490760,10.0,1745.0,103017.0,28.0,NaN,...,28.0,1130.0,W/O,5.0,R16,NaN,NaN,NaN,NaN,NaN
3003,Nicolas Massu,R,183.0,CHI,25.886379,58.0,665.0,102562.0,23.0,NaN,...,25.0,1230.0,W/O,5.0,R64,NaN,NaN,NaN,NaN,NaN
3922,Roger Federer,R,185.0,SUI,25.878166,1.0,7290.0,103163.0,13.0,NaN,...,10.0,1960.0,W/O,5.0,R16,NaN,NaN,NaN,NaN,NaN
5044,Jo Wilfried Tsonga,R,188.0,FRA,24.180698,9.0,3510.0,104620.0,NaN,NaN,...,63.0,951.0,W/O,5.0,R64,NaN,NaN,NaN,NaN,NaN
5166,Gilles Simon,R,183.0,FRA,25.481177,32.0,1305.0,105011.0,NaN,NaN,...,74.0,672.0,W/O,5.0,R64,NaN,NaN,NaN,NaN,NaN
5435,Lukasz Kubot,R,190.0,POL,27.676934,86.0,573.0,104022.0,20.0,NaN,...,20.0,1690.0,W/O,5.0,R32,NaN,NaN,NaN,NaN,NaN
6092,Novak Djokovic,R,188.0,SRB,24.002738,2.0,11665.0,104926.0,NaN,NaN,...,49.0,960.0,W/O,5.0,QF,NaN,NaN,NaN,NaN,NaN


In [21]:
data[data['minutes'].isna()].loc[:,data.columns[10:35]].iloc[0]

winner_name           Rainer Schuettler
winner_hand                           R
winner_ht                         180.0
winner_ioc                          GER
winner_age                    26.718686
winner_rank                        36.0
winner_rank_points                945.0
loser_id                       103498.0
loser_seed                          3.0
loser_entry                         NaN
loser_name                  Marat Safin
loser_hand                            R
loser_ht                          193.0
loser_ioc                           RUS
loser_age                     22.962355
loser_rank                          3.0
loser_rank_points                2860.0
score                               W/O
best_of                             5.0
round                               R32
minutes                             NaN
w_ace                               NaN
w_df                                NaN
w_svpt                              NaN
w_1stIn                             NaN


In [15]:
data.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'winner_rank', 'winner_rank_points', 'loser_id', 'loser_seed',
       'loser_entry', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'loser_rank', 'loser_rank_points', 'score', 'best_of',
       'round', 'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon',
       'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
       'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced'],
      dtype='object')

In [103]:
for i in range(100):
    print(data['tourney_name'][i])
    print(data['score'][i].split(' '))

US Open
['6-4', '6-2', '6-0']
US Open
['6-3', '6-4', '6-2']
US Open
['6-3', '6-0', '4-6', '6-7(3)', '6-4']
US Open
['6-3', '2-6', '6-4', '6-1']
US Open
['6-4', '6-4', '4-6', '6-3']
US Open
['6-3', '6-4', '6-3']
US Open
['6-2', '6-2', '6-2']
US Open
['7-5', '6-4', '7-6(4)']
US Open
['2-6', '7-5', '6-3', '6-3']
US Open
['7-6(3)', '6-3', '6-0']
US Open
['6-4', '6-4', '6-4']
US Open
['7-6(7)', '6-3', '7-5']
US Open
['6-3', '7-5', '1-6', '7-6(6)']
US Open
['6-4', '6-3', '6-3']
US Open
['6-2', '6-3', '6-0']
US Open
['3-6', '6-3', '6-2', '6-2']
US Open
['7-6(3)', '7-5', '6-4']
US Open
['3-6', '6-3', '6-4', '7-6(4)']
US Open
['7-5', '6-4', '6-2']
US Open
['7-6(4)', '7-6(3)', '6-2']
US Open
['6-3', '6-1', '6-1']
US Open
['7-6(4)', '6-2', '6-4']
US Open
['6-3', '3-6', '6-3', '7-6(6)']
US Open
['6-3', '6-3', '7-6(5)']
US Open
['6-3', '6-3', '6-4']
US Open
['6-3', '6-4', '6-1']
US Open
['6-7(1)', '6-1', '6-1', '6-4']
US Open
['3-6', '6-4', '6-1', '7-6(4)']
US Open
['3-6', '6-0', '6-1', '6-0']
US O

In [ ]:
dat

In [93]:
data['score'][0].split(' ')

['6-4', '6-2', '6-0']

In [43]:
data['draw_size']

array([128])

In [39]:
set(data.tourney_name)

{'Australian Open', 'Roland Garros', 'US Open', 'Wimbledon'}

In [65]:
data[data.winner_name=='Rafael Nadal'][ 'winner_entry'].dropna()

Series([], Name: winner_entry, dtype: object)

In [66]:
data[data.winner_entry=='Q']

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
19,2000-560,US Open,Hard,128,G,20000828.0,20.0,103457,NaN,Q,...,15.0,4.0,2.0,147.0,92.0,53.0,26.0,16.0,12.0,18.0
22,2000-560,US Open,Hard,128,G,20000828.0,23.0,102703,NaN,Q,...,10.0,10.0,11.0,161.0,100.0,67.0,26.0,19.0,15.0,20.0
25,2000-560,US Open,Hard,128,G,20000828.0,26.0,103602,NaN,Q,...,6.0,0.0,2.0,74.0,45.0,24.0,8.0,13.0,5.0,13.0
46,2000-560,US Open,Hard,128,G,20000828.0,47.0,102642,NaN,Q,...,3.0,5.0,5.0,95.0,38.0,29.0,27.0,15.0,3.0,8.0
60,2000-560,US Open,Hard,128,G,20000828.0,61.0,103598,NaN,Q,...,6.0,5.0,14.0,103.0,52.0,39.0,23.0,16.0,8.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7893,2015-580,Australian Open,Hard,128,G,20150119.0,20.0,104897,NaN,Q,...,3.0,8.0,5.0,100.0,63.0,41.0,13.0,14.0,5.0,10.0
7907,2015-580,Australian Open,Hard,128,G,20150119.0,34.0,103781,NaN,Q,...,5.0,1.0,1.0,73.0,44.0,24.0,12.0,12.0,3.0,9.0
7920,2015-580,Australian Open,Hard,128,G,20150119.0,47.0,105065,NaN,Q,...,7.0,5.0,4.0,119.0,74.0,46.0,23.0,17.0,8.0,13.0
8019,2015-520,Roland Garros,Clay,128,G,20150524.0,19.0,106378,NaN,Q,...,18.0,7.0,4.0,144.0,82.0,54.0,24.0,23.0,6.0,15.0


In [62]:
set(data[data.winner_seed<2][ 'winner_name'])

{'Andre Agassi',
 'Andy Roddick',
 'Gustavo Kuerten',
 'Lleyton Hewitt',
 'Novak Djokovic',
 'Pete Sampras',
 'Rafael Nadal',
 'Roger Federer'}

In [54]:
data[data.winner_seed==2]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
190,2000-580,Australian Open,Hard,128,G,20000117.0,64.0,102338,2.0,NaN,...,5.0,20.0,3.0,136.0,65.0,48.0,29.0,19.0,9.0,16.0
222,2000-580,Australian Open,Hard,128,G,20000117.0,96.0,102338,2.0,NaN,...,5.0,3.0,7.0,79.0,42.0,26.0,8.0,11.0,6.0,14.0
238,2000-580,Australian Open,Hard,128,G,20000117.0,112.0,102338,2.0,NaN,...,0.0,14.0,4.0,76.0,44.0,34.0,16.0,14.0,3.0,7.0
246,2000-580,Australian Open,Hard,128,G,20000117.0,120.0,102338,2.0,NaN,...,12.0,3.0,9.0,95.0,54.0,35.0,17.0,14.0,5.0,10.0
250,2000-580,Australian Open,Hard,128,G,20000117.0,124.0,102338,2.0,NaN,...,0.0,13.0,2.0,89.0,58.0,36.0,15.0,14.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7969,2015-580,Australian Open,Hard,128,G,20150119.0,96.0,103819,2.0,NaN,...,1.0,8.0,3.0,109.0,57.0,36.0,28.0,17.0,6.0,11.0
8064,2015-520,Roland Garros,Clay,128,G,20150524.0,64.0,103819,2.0,NaN,...,2.0,2.0,4.0,109.0,74.0,48.0,16.0,14.0,12.0,16.0
8096,2015-520,Roland Garros,Clay,128,G,20150524.0,96.0,103819,2.0,NaN,...,3.0,2.0,3.0,94.0,62.0,38.0,14.0,14.0,3.0,8.0
8112,2015-520,Roland Garros,Clay,128,G,20150524.0,112.0,103819,2.0,NaN,...,5.0,3.0,2.0,81.0,57.0,32.0,12.0,14.0,2.0,8.0
